In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [2]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [3]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (파일 이름으로 자동 탐색)
def find_file(filename, search_path='.'):
    # 주어진 파일명을 search_path부터 재귀적으로 탐색하여 경로 반환
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# 상위 디렉토리부터 탐색 시작
# 현재 노트북의 상위 디렉토리부터 탐색 시작
# base_path = pathlib.Path.cwd().parent.parent
# 현재 디렉토리부터 탐색 시작
base_path = pathlib.Path.cwd()

card_writer_schema_path = find_file("CardWriter_Object.json", base_path)
card_writer_system_prompt_path = find_file("CardWriter_SystemPrompt.md", base_path)

if not card_writer_schema_path or not card_writer_system_prompt_path:
    raise FileNotFoundError("Card Writer 프롬포트 파일을 찾을 수 없습니다")

with open(card_writer_schema_path, 'r', encoding='utf-8') as f:
    card_writer_schema = json.load(f)

with open(card_writer_system_prompt_path, 'r', encoding='utf-8') as f:
    card_writer_system_prompt = f.read()

In [5]:
# 입력: (강의 자료, concept_plan, current_feedback, target_card_count, user_profile, prior_content)
lecture_material = "/Users/jhkim/Desktop/LectureTestGenerator/SamSung(260109).pdf"
concept_plan = {
    "planning_strategy": "학습자의 'Intermediate' 수준과 'Deep Understanding' 목표를 고려하여, 단순히 실적 수치를 나열하기보다는 각 사업부별 실적 변동의 '인과관계(Causal Relationship)'와 향후 시장 전망에 대한 '시나리오(Application_Scenario)' 분석을 중심으로 10개의 카드를 구성함.",
    "planned_items": [
        {
            "id": 1,
            "target_topic": "2025년 4분기 실적 총평",
            "intent_type": "Definition",
            "complexity_level": "Basic",
            "source_reference_hint": "매출액 93조원(전분기 대비 8.0% 증가) 및 영업이익 20.3조원을 기록했으나, 이전 전망치보다는 감소한 규모임."
        },
        {
            "id": 2,
            "target_topic": "DS(Device Solutions) 부문 영업이익 개선 요인",
            "intent_type": "Causal_Relationship",
            "complexity_level": "Intermediate",
            "source_reference_hint": "DRAM 및 SSD 수요 증가에 따른 가격 상승(ASP 상승) 효과와 HBM 물량 증가가 주요 원인임."
        },
        {
            "id": 3,
            "target_topic": "MX(Mobile eXperience) 부문 수익성 악화 원인",
            "intent_type": "Causal_Relationship",
            "complexity_level": "Intermediate",
            "source_reference_hint": "역설적으로 메모리 가격 상승이 원가 구조를 악화시켜 예상치 대비 실적이 크게 부진함."
        },
        {
            "id": 4,
            "target_topic": "VD/가전 사업부의 적자 발생 배경",
            "intent_type": "Application_Scenario",
            "complexity_level": "Intermediate",
            "source_reference_hint": "내구재 소비 부진, 업계 경쟁 심화, 그리고 원재료 가격 상승이라는 복합적 리스크가 작용함."
        },
        {
            "id": 5,
            "target_topic": "25년 4분기 DRAM vs NAND 출하량(Shipment) 비교",
            "intent_type": "Comparison",
            "complexity_level": "Intermediate",
            "source_reference_hint": "DRAM은 낮은 한자리수 증가를 기록한 반면, NAND는 10% 감소하며 상반된 흐름을 보임."
        },
        {
            "id": 6,
            "target_topic": "HBM4(High Bandwidth Memory 4) 시장 전망",
            "intent_type": "Application_Scenario",
            "complexity_level": "Advanced",
            "source_reference_hint": "2026년에는 HBM 관련 우려가 완화되며 HBM4 시장에서 의미 있는 점유율 확보가 가능할 것으로 전망함."
        },
        {
            "id": 7,
            "target_topic": "목표주가 180,000원 산출 근거",
            "intent_type": "Causal_Relationship",
            "complexity_level": "Intermediate",
            "source_reference_hint": "2026년 예상 BPS(78,295원)에 PBR 2.3배를 적용하여 산출함."
        },
        {
            "id": 8,
            "target_topic": "2026년 영업이익 전망치 급증 사유",
            "intent_type": "Definition",
            "complexity_level": "Advanced",
            "source_reference_hint": "AI 투자 확대에 따른 메모리 수요 본격화로 2025년(약 43.8조원) 대비 2026년(약 151.2조원) 영업이익이 대폭 증가할 것으로 예상함."
        },
        {
            "id": 9,
            "target_topic": "메모리 가격 상승이 전사 수익성에 미치는 양면성",
            "intent_type": "Comparison",
            "complexity_level": "Advanced",
            "source_reference_hint": "DS 부문에는 이익 증가 요인이지만, MX 및 VD/가전 부문에는 원가 부담을 가중시켜 수익성을 저해함."
        },
        {
            "id": 10,
            "target_topic": "ROE(Return on Equity) 변화 추이 분석",
            "intent_type": "Causal_Relationship",
            "complexity_level": "Intermediate",
            "source_reference_hint": "2024년 9.0%에서 2026년 29.0%까지 급상승할 것으로 추정되며 이는 수익성 개선의 지표로 활용됨."
        }
    ]
}
current_feedback = None
target_card_count = 10
user_profile = {
    "learning_goal": {
        "focus_areas": [
            "삼성전자 실적",
            "삼성전자 전망"
        ],
        "target_depth": "Deep Understanding",
        "question_modality": "Conceptual"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": "Korean_with_Korean_Terms",
        "scenario_based": True
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}
prior_content = None

# lecture_material 파일 읽기
path = pathlib.Path(lecture_material)

# 파일 타입별 처리
if path.is_file(): # 실제 존재하는 파일인지 먼저 검사
    # 파일 경로 소문자로 전환 (일관성 높이기 위해)
    suffix = path.suffix.lower()
    # PDF인 경우 처리
    if suffix == '.pdf':
        lecture_content = client.files.upload(file=path)
    # 텍스트 파일인 경우 처리
    elif suffix in ['.txt', '.md']:
        lecture_content = path.read_text(encoding='utf-8')
    # 지원되지 않는 파일은 경우
    else:
        lecture_content = "강의 자료 없음"
else:
    # 파일이 없다면, 파일 경로가 아닌, 강의 텍스트가 입력되었다고 처리
    lecture_content = lecture_material


# contents 생성
cardWriter_contents = [
    lecture_content,
    "Concept Plan: " + json.dumps(concept_plan),
    "Current Feedback: " + str(current_feedback),
    "Target Card Count: " + str(target_card_count),
    "User Profile: " + json.dumps(user_profile),
    "Prior Content: " + json.dumps(prior_content)
]


In [6]:
# Response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = cardWriter_contents,
    config={
        "system_instruction": card_writer_system_prompt,
        "response_mime_type": "application/json",
        "response_schema": card_writer_schema
    })

In [7]:
print(response.text)

{
  "flash_cards": [
    {
      "id": 1,
      "front_content": "삼성전자의 2025년 4분기 잠정 매출액은 93조 원, 영업이익은 20.3조 원을 기록하며 전분기 대비 성장했습니다. 그러나 이 실적 규모가 시장의 이전 전망치와 비교했을 때 어떤 평가를 받았으며, 그 주된 이유는 무엇입니까?",
      "back_content": "이전 전망 대비 감소한 규모로 평가받았습니다. 이는 DS와 디스플레이는 예상 수준이었으나, MX 및 VD/가전 사업부의 수익성이 경기 부진과 원가 상승으로 인해 전망보다 크게 부진했기 때문입니다.",
      "category_tag": "2025 4Q 실적 총평"
    },
    {
      "id": 2,
      "front_content": "2025년 4분기 실적 개선의 핵심 동력이었던 DS(Device Solutions) 부문의 영업이익 증가(전분기 대비 10조 원 증가)를 이끈 구체적인 제품군과 가격 관련 요인은 무엇입니까?",
      "back_content": "DRAM과 SSD의 수요 증가에 따른 가격 상승(ASP 상승) 효과와 더불어 HBM 물량 증가가 영업이익 개선의 주요 원인이었습니다.",
      "category_tag": "DS 사업부"
    },
    {
      "id": 3,
      "front_content": "[시나리오] 당신은 경영 전략 회의에 참석 중입니다. MX(모바일) 사업부의 실적이 예상보다 크게 부진한 원인에 대해 질문을 받았을 때, '메모리 시장 상황'과 연계하여 어떻게 답변하시겠습니까?",
      "back_content": "메모리 반도체 가격의 상승으로 인해 스마트폰 제조를 위한 원가 구조가 악화된 것이 MX 사업부 수익성 부진의 주요 원인이라고 답변해야 합니다.",
      "category_tag": "MX 사업부"
    },
    {
      "id": 4,
      "front_